In [15]:
#!pip install datasets
#!pip uninstall fsspec -y
#!pip install fsspec
!pip install seqeval

     |████████████████████████████████| 43 kB 201 kB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=53888f8f1c59fb1bc55f4b8d949da781d9aba11fba9901a47835c06418c242d5
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


**Imports**

In [5]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric
import torch
from tqdm.auto import tqdm
import numpy as np

In [3]:
task = 'ner'
model_checkpoint = 'bert-base-uncased'
batch_size = 16

**Load tokenizer, dataset**

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
dataset = load_dataset("conll2003")

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


**conll2003**

**id**: a string feature.

**tokens**: a list of string features.

**pos_tags**: a list of classification labels, with possible values including " (0), '' (1), # (2), $ (3), ( (4).

**chunk_tags**: a list of classification labels, with possible values including O (0), B-ADJP (1), I-ADJP (2), B-ADVP (3), I-ADVP (4).

**ner_tags**: a list of classification labels, with possible values including O (0), B-PER (1), I-PER (2), B-ORG (3), I-ORG (4) B-LOC (5), I-LOC (6) B-MISC (7), I-MISC (8).

In [ ]:
print(dataset)

In [8]:
print(dataset['train']['tokens'][0])
print(dataset['train']['ner_tags'][0])
print(dataset['train'].features['ner_tags'])
label_list = dataset['train'].features['ner_tags'].feature.names
print(label_list)

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[3, 0, 7, 0, 0, 0, 7, 0, 0]
Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)
['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [9]:
def tokenize_and_align_labels(examples):
    tokens = examples['tokens']
    tokenized_inputs = tokenizer(tokens, truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[task+'_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # for each token it tells the word it belongs to
        label_ids = []  # the label_id (integer) corresponsing to each token in tokens
        for word_idx in word_ids:
            if word_idx is None:  # then it is a special token like [CLS] or [PAD]
                label_ids.append(-100)  # such that loss function ignores it
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)
    tokenized_inputs['labels'] = labels  # label_ids for each example is of diff length. we'll pad using datacollator later
    return tokenized_inputs

tokenize train and test at once by calling the map function in datasets library on the dataset object

In [10]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# **Finetuning**

In [11]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [12]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

the data collator will batch the data and also pad both inputs and labels

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
metric = load_metric('seqeval')

In [17]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [18]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [19]:
trainer.train()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Runtime,Samples Per Second
1,0.218500,0.062283,0.927749,0.927956,0.927852,0.982589,17.240200,188.513000
2,0.047500,0.056785,0.934957,0.940709,0.937824,0.985210,16.388000,198.316000
3,0.025000,0.059102,0.933017,0.945855,0.939392,0.985893,14.944700,217.469000


TrainOutput(global_step=2634, training_loss=0.07709464650309801, metrics={'train_runtime': 1125.5856, 'train_samples_per_second': 2.34, 'total_flos': 1305657034593678.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1595219968, 'init_mem_gpu_alloc_delta': 436731392, 'init_mem_cpu_peaked_delta': 315928576, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 206180352, 'train_mem_gpu_alloc_delta': 1316473856, 'train_mem_cpu_peaked_delta': 245383168, 'train_mem_gpu_peaked_delta': 2475994112})

In [20]:
trainer.evaluate()

{'eval_loss': 0.05910233408212662,
 'eval_precision': 0.9330169940410505,
 'eval_recall': 0.9458552410784205,
 'eval_f1': 0.9393922559857786,
 'eval_accuracy': 0.985892894021955,
 'eval_runtime': 15.7199,
 'eval_samples_per_second': 206.745,
 'epoch': 3.0,
 'eval_mem_cpu_alloc_delta': 155648,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 2867200,
 'eval_mem_gpu_peaked_delta': 93145600}